# ++ Carregar dados históricos da Bovespa em Dataframe para análise ++

In [1]:
import time
time.asctime( time.localtime(time.time()) )

'Thu Nov 17 15:39:00 2022'

In [2]:
import sys
from pandas import Series
import pandas as pd
from datetime import date
pd.__version__
from zipfile import ZipFile

In [3]:
import matplotlib as mat
mat.__version__

'3.4.3'

In [4]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [5]:
from IPython.display import FileLink, FileLinks
FileLink('Manipulação CSV BB.ipynb')

C:\Users\F0435280\OneDrive - Banco do Brasil S.A\Área de Trabalho\Alexandre\StockManager-main\Manipulação CSV BB.ipynb

In [6]:
# Setando opção do Pandas para mostrar todas as linhas
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [7]:
# Setando separador de milhar
pd.options.display.float_format = '{:,.2f}'.format

In [8]:
def leituraArquivos(periodo):
    arq = 'COTAHIST_' + periodo

    DTEXCH, CODNEG, PREABE, PREMAX, PREMIN, PREULT, VOLTOT = ([] for i in range(7)) 
    
    with ZipFile('arquivos/' + arq + '.ZIP') as myzip:
        with myzip.open(arq + '.TXT') as myfile:
            for line in myfile:
                if (int(line.decode('utf-8')[0:2]) == 1) and (line.decode('utf-8')[10:12] == '02'):
                    DTEXCH.append(line.decode('utf-8')[2:10])
                    CODNEG.append(line.decode('utf-8')[12:24].rstrip())
                    PREABE.append(int(line.decode('utf-8')[56:69]) / 100)
                    PREMAX.append(int(line.decode('utf-8')[69:82]) / 100)
                    PREMIN.append(int(line.decode('utf-8')[82:95]) / 100)
                    PREULT.append(int(line.decode('utf-8')[108:121]) / 100)
                    VOLTOT.append(int(line.decode('utf-8')[170:188]) / 100)

    df_origem = pd.DataFrame(
        {"cdAcao":CODNEG
        ,"dtPregao":pd.to_datetime(DTEXCH, format="%Y%m%d", errors="ignore")
        ,"vrFech":PREULT
        ,"vrVolume":VOLTOT
        ,"vrMaxDia":PREMAX
        ,"vrMinDia":PREMIN
        ,"vrAbert":PREABE
        }
    ).sort_values(["cdAcao", "dtPregao"], ascending=True)

    return(df_origem)

In [9]:
def carrega_dados():
    df = leituraArquivos('A2022')

#    df = pd.concat([df_112021, df_122021, df])

    df["pcVar"], df["pcMaxDia"], df["pcMinDia"], df["pcAbert"] = [
         ((df.vrFech   / df.vrFech.shift(1)) -1) *100
        ,((df.vrMaxDia / df.vrFech.shift(1)) -1) *100
        ,((df.vrMinDia / df.vrFech.shift(1)) -1) *100
        ,((df.vrAbert  / df.vrFech.shift(1)) -1) *100
        ]
    
    df["ic05"], df["ic10"], df["ic15"], df["ic20"], df["ic25"], df["ic30"] = [
         df.apply(condicao05, axis=1)
        ,df.apply(condicao10, axis=1)
        ,df.apply(condicao15, axis=1)
        ,df.apply(condicao20, axis=1)
        ,df.apply(condicao25, axis=1)
        ,df.apply(condicao30, axis=1)
        ]

    return df

In [10]:
def condicao05(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 0.5) else 0
def condicao10(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 1) else 0
def condicao15(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 1.5) else 0
def condicao20(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 2) else 0
def condicao25(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 2.5) else 0
def condicao30(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 3) else 0

In [11]:
# Exclui header e trailer
# df = df.drop(df.index[[0, -1]])

In [12]:
def buscaPeriodos(df, qt_dias):
    return df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[qt_dias])].sort_values(["cdAcao", "dtPregao"], ascending=False)

In [13]:
def somatorioPcMaxDia(df_ent, pc, index_name):
    return df_ent.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>pc).sum()).reset_index(name=index_name)

In [14]:
def buscaMedia(df_ent, coluna, index_name):
    return df_ent.groupby("cdAcao")[coluna].agg("mean").reset_index(name=index_name)

In [15]:
df = carrega_dados()

In [16]:
def montaDfPeriodos(df_origem, qt_dias):
    df_dias  = buscaPeriodos(df_origem, qt_dias-1)
    
    df05 = somatorioPcMaxDia(df_dias, 0.5, "0.5%")
    df10 = somatorioPcMaxDia(df_dias, 1.0, "resultado")
    df15 = somatorioPcMaxDia(df_dias, 1.5, "resultado")
    df20 = somatorioPcMaxDia(df_dias, 2.0, "resultado")
    df25 = somatorioPcMaxDia(df_dias, 2.5, "resultado")
    df30 = somatorioPcMaxDia(df_dias, 3.0, "resultado")
    dfVol     = buscaMedia(df_dias, "vrVolume", "vol")
    dfVrFech  = buscaMedia(df_dias, "vrFech", "vrFech")
    dfPcAbert = buscaMedia(df_dias, "pcAbert", "pcAbert")
    
    df05["1.0%"], df05["1.5%"], df05["2.0%"], df05["2.5%"], df05["3.0%"] , df05["AvgVol"], df05["AvgVrFech"], df05["AvgPcAbert"] = [
    df10["resultado"],df15["resultado"],df20["resultado"],df25["resultado"],df30["resultado"],dfVol["vol"],dfVrFech["vrFech"],dfPcAbert["pcAbert"]]

    df_result = df05.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)

    return df_result

### 25 dias, order by 1%

In [32]:
montaDfPeriodos(df, 25).loc[(df_25dias["AvgVol"] > 6000000) &
                            (df_25dias["1.0%"] >= 21) & (df_25dias["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
202,GFSA3,23,21,19,16,15,13,"13,416,814.80",7.61,0.09
168,EMBR3,21,21,18,13,11,8,"106,979,306.32",12.95,-0.14
169,ENAT3,21,21,18,10,8,6,"23,553,387.68",15.28,-0.47
326,PTBL3,21,21,17,13,10,8,"16,279,416.16",11.98,-0.37
241,KLBN11,24,21,15,13,10,8,"148,011,552.20",21.36,-0.01


### 20 dias, order by 1%

In [27]:
montaDfPeriodos(df, 20).loc[(df_20dias["AvgVol"] > 6000000) & 
                            (df_20dias["1.0%"] >= 17) & (df_20dias["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
340,ROMI3,19,18,16,12,11,7,"13,036,723.85",16.07,-0.01
167,ENAT3,18,18,15,9,7,5,"24,638,488.90",15.24,-0.53
200,GFSA3,19,17,16,13,13,11,"14,560,341.25",7.54,0.05
424,YDUQ3,18,17,16,13,12,9,"125,913,791.10",13.84,0.21
326,QUAL3,18,17,15,13,9,9,"39,321,702.05",7.38,-0.23
166,EMBR3,17,17,14,11,9,7,"111,734,264.95",13.19,-0.23
140,CVCB3,18,17,13,12,10,8,"100,899,931.25",6.63,-0.21


### 15 dias, order by 1%

In [29]:
montaDfPeriodos(df, 15).loc[(df_15dias["AvgVol"] > 6000000) & 
                            (df_15dias["1.5%"] >= 12) & (df_15dias["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
198,GFSA3,14,14,13,11,11,10,"15,700,533.60",7.64,-0.13
333,ROMI3,14,14,12,9,9,6,"13,967,243.73",16.13,-0.00
417,YDUQ3,14,13,13,10,10,7,"138,126,958.20",14.23,0.29
319,QUAL3,13,13,12,11,8,8,"38,691,901.87",7.38,-0.27
46,BLAU3,13,13,12,9,8,6,"12,693,593.53",32.19,0.33
410,VVEO3,13,13,12,9,8,5,"8,145,614.33",17.81,-0.79
139,CVCB3,13,12,12,11,9,7,"107,320,476.87",6.55,-0.24
251,LWSA3,12,12,12,9,8,7,"113,160,319.93",9.68,-1.03


### 10 dias, order by 1%

In [30]:
montaDfPeriodos(df, 10).loc[(df_10dias["AvgVol"] > 6000000) & 
                            (df_10dias["1.0%"] >= 9) & (df_10dias["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
312,QUAL3,9,9,9,8,6,6,"43,148,875.00",7.39,-0.23
134,CVCB3,9,9,9,8,6,5,"110,855,782.50",6.53,0.00
193,GFSA3,9,9,9,7,7,6,"15,720,216.30",7.72,-0.25
326,ROMI3,9,9,9,6,6,3,"11,281,459.80",16.21,0.30
318,RAPT4,9,9,8,6,4,2,"21,884,389.50",9.86,-0.03
401,VVEO3,9,9,8,5,4,2,"11,169,692.00",17.57,-0.50
385,UGPA3,9,9,7,7,3,3,"140,930,408.30",13.70,-0.45
357,SULA11,10,9,7,6,5,4,"96,129,514.30",23.97,-0.15
360,SUZB3,10,9,7,6,5,3,"407,097,270.60",55.78,0.25
409,ZAMP3,9,9,6,6,5,3,"11,714,776.00",6.85,-0.25


### 5 dias, order by 1%

In [31]:
montaDfPeriodos(df, 5).loc[(df_5dias["AvgVol"] > 6000000) & 
                           (df_5dias["1.0%"] >= 5) & (df_5dias["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
186,GGBR4,5,5,5,3,2,2,"616,959,946.60",29.99,0.51
152,EMBR3,5,5,4,3,3,3,"189,888,457.60",13.64,-0.28
351,SUZB3,5,5,3,3,3,2,"512,174,591.60",57.22,0.12
348,SULA11,5,5,3,2,2,1,"113,029,241.80",23.45,-0.01


### Consulta ação específica

In [ ]:
df25d.loc[(df25d["cdAcao"] == "GFSA3")].replace(0, "")

In [ ]:
# dfDtMin = df5d.loc[(df["dtPregao"] == dt_min_5d) & (df["cdAcao"] == 'PTBL3')]
dt_min_5d  = df5d["dtPregao"].min()
dt_min_25d = df25d["dtPregao"].min()
dt_max     = df5d["dtPregao"].max()
dfDtMin5d  = df5d.loc[(df5d["dtPregao"] == dt_min_5d)].set_index(["cdAcao"])
dfDtMin25d = df25d.loc[(df25d["dtPregao"] == dt_min_25d)].set_index(["cdAcao"])
dfDtMax    = df5d.loc[(df5d["dtPregao"] == dt_max)].set_index(["cdAcao"])

In [ ]:
 dfPc5d = pd.DataFrame({
         "dtInicio":dfDtMin5d["dtPregao"] ,"dtFim":dfDtMax["dtPregao"]
        ,"vrInicio":dfDtMin5d["vrFech"]   ,"vrFim":dfDtMax["vrFech"]
        ,"pcPeriodo":((dfDtMax["vrFech"] - dfDtMin5d["vrFech"]) / dfDtMin5d["vrFech"]) * 100
    })

In [ ]:
dfPc5d.sort_values(["pcPeriodo"], ascending=False)

In [ ]:
 dfPc25d = pd.DataFrame({
         "dtInicio":dfDtMin25d["dtPregao"] ,"dtFim":dfDtMax["dtPregao"]
        ,"vrInicio":dfDtMin25d["vrFech"]   ,"vrFim":dfDtMax["vrFech"]
        ,"pcPeriodo":((dfDtMax["vrFech"] - dfDtMin25d["vrFech"]) / dfDtMin25d["vrFech"]) * 100
    })

In [ ]:
dfPc25d.sort_values(["pcPeriodo"], ascending=False)

In [ ]:
### 25 dias, Order by 0.5% # df05.loc[(df05["AvgVol"] > 6000000) & (df05["0.5%"] >= 23) & (df05["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 20 dias, Order by 0.5% # df05_20d.loc[(df05_20d["AvgVol"] > 6000000) & (df05_20d["0.5%"] >= 19) & (df05_20d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 15 dias, Order by 0.5% # df05_15d.loc[(df05_15d["AvgVol"] > 6000000) & (df05_15d["0.5%"] >= 15) & (df05_15d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)

## Definindo valores para compra

In [ ]:
# Valor disponível para cmp / 1000
# Valor da ação no leilão
vr_corretagem = 9.8
emolumentos = 0.000325
acao = 'LIGT3'
vr_disp = 110
vr_acao_leilao = 1148 / 100
vr_qtd = round((vr_disp * 1000) / (vr_acao_leilao * 100)) * 100

In [ ]:
vr_cmp = round(vr_acao_leilao * 1.014, 2)

vr_tot_cmp = vr_acao_leilao * vr_qtd
corret_cmp = vr_corretagem + (vr_tot_cmp * emolumentos)
vr_tot_liq_cmp = vr_tot_cmp + corret_cmp

vr_acao_vnd_10 = float('{0:.2f}'.format(vr_acao_leilao * 1.0108 + 0.01))
vr_tot_vnd_10 = vr_acao_vnd_10 * vr_qtd
corret_vnd_10 = vr_corretagem + (vr_tot_vnd_10 * emolumentos)
vr_tot_liq_vnd_10 = vr_tot_vnd_10 - corret_vnd_10

vr_acao_vnd_15 = float('{0:.2f}'.format(vr_acao_leilao * 1.0158 + 0.01))
vr_tot_vnd_15 = vr_acao_vnd_15 * vr_qtd
corret_vnd_15 = vr_corretagem + (vr_tot_vnd_15 * emolumentos)
vr_tot_liq_vnd_15 = vr_tot_vnd_15 - corret_vnd_15

In [ ]:
d = {'Compra':     ['Quantidade', 'Valor Leilão', 'Total', 'Total líquido', 'Corretagem Compra'],
     ' ':          [vr_qtd, vr_acao_leilao, vr_tot_cmp, vr_tot_liq_cmp, corret_cmp],
     'Venda 1,0%': ['Valor compra', 'Valor', 'Total', 'Total líquido', 'Corretagem Venda'],
     '1,0%':       [vr_cmp, vr_acao_vnd_10, vr_tot_vnd_10, vr_tot_liq_vnd_10, corret_vnd_10],
     'Venda 1,5%': [' ', 'Valor', 'Total', 'Total líquido', 'Corretagem Venda'],
     '1,5%':       [' ', vr_acao_vnd_15, vr_tot_vnd_15, vr_tot_liq_vnd_15, corret_vnd_15]
    }
df_calc = pd.DataFrame(data=d)
df_calc

In [ ]:
d2 = {'Index':      ['Quantidade', 'Valor', 'Total', 'Total líquido', 'Corretagem'],
      'Compra':     [vr_qtd, vr_cmp, vr_tot_cmp, vr_tot_liq_cmp, corret_cmp],
      'Venda 1.0%': [vr_qtd, vr_acao_vnd_10, vr_tot_vnd_10, vr_tot_liq_vnd_10, corret_vnd_10],
      'Venda 1.5%': [vr_qtd, vr_acao_vnd_15, vr_tot_vnd_15, vr_tot_liq_vnd_15, corret_vnd_15]
     }
df_calc2 = pd.DataFrame(data=d2)
df_calc2.set_index('Index')
df_calc2

In [ ]:
time.asctime( time.localtime(time.time()) )